Снова начнем с загрузки графа.

In [ ]:
import json
full_friends = json.load(open("nodes.txt", "tr"))
full_graph = json.load(open("edges.txt", "tr"))
import networkx as nx
G = nx.Graph()
for i in full_friends["items"]:
    G.add_node(i["id"], name = i["first_name"]+" "+i["last_name"], sex = i["sex"])
my_friends = list(nx.nodes(G))
for i in my_friends:
    if "items" in full_graph[str(i)]:
        for j in full_graph[str(i)]["items"]:
            if j in my_friends:
                G.add_edge(i, j)



Немного изменим функцию отрисовки графа.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
def plot_graph(g, coloring = [], palette = plt.cm.Set2):
    with plt.xkcd():
        k = nx.degree(g)
        plt.figure(1, figsize=(60,45))
        coord = nx.kamada_kawai_layout(g)
        labels={nd: g.node[nd]['name'] for (nd) in g.nodes()}
        if len(coloring)>0:
            nx.draw_networkx(g, pos=coord, nodelist=dict(k).keys(), node_size=[v*50 for v in dict(k).values()], 
                         font_size=17, node_color=coloring, labels=labels, cmap=palette)
        else:
            nx.draw_networkx(g, pos=coord, nodelist=dict(k).keys(), node_size=[v*50 for v in dict(k).values()], 
                         font_size=17, labels=labels)
            

Для начала оставим только самую большую компоненту связности.

In [ ]:
largest_cc = max(nx.connected_components(G), key=len)
Gmain = G.subgraph(largest_cc)
plot_graph(Gmain)

Начнем с задачи бисекции графа. Задача состоит в том, чтобы разделить граф на две части, так чтобы минимизировать число связей между этими частями. Очевидно, что полный перебор всех разделений неэффективен.

Пусть у нас уже есть две компоненты, давайте рассмотрим все возможные пары вершин и посмотрим, станит ли разделение лучше, если мы поменяем две вершины местами. Такой алгоритм называется алгоритм Кернигана-Лина. Он может быть обобщен и более чем на две компоненты.

In [ ]:
from networkx.algorithms import community
TwoParts = community.kernighan_lin_bisection(Gmain)
coloring = [0 if x in TwoParts[0] else 1 for x in Gmain.nodes]

In [ ]:
plot_graph(Gmain, coloring)

По умолчанию граф делится на две равные части. Давайте предложим ему альтернативное начальное разделение и посмотрим сможет ли алгоритм что-то сделать. 

In [ ]:
InitialTwoParts = [set(list(Gmain.nodes)[:50]), set(list(Gmain.nodes)[50:])]

In [ ]:
TwoParts2 = community.kernighan_lin_bisection(Gmain, InitialTwoParts)
coloring2 = [0 if x in TwoParts2[0] else 1 for x in Gmain.nodes]

In [ ]:
plot_graph(Gmain, coloring2)

Хотелось выделить "центральную компоненту", но начальное распределение оказалось слишком размытым и не сошлось к чему-то более пристойному. Да и в целом алглоритм достаточно медленный порядка $O(n^3)$ для разреженных сетей и $O(n^4)$ для плотных.

Рассмотрим альтернативный подход, который называется спектральное разделение.
Для этого построим Лапласиан для графа: $L = D - A$, где $D$ диагональная матрица степеней вершин, а $A$ -- матрица смежности.



In [ ]:
import numpy as np
import numpy.linalg as la
A = nx.adjacency_matrix(Gmain)
D = np.diag(np.ravel(np.sum(A,axis=1)))
L=D-A
l, U = la.eigh(L)
f = U[:,1]

Мы только что нашли собственный вектор соотвествуйщий второму по величине собственному числу Лапласиана. При этом если нам нужно выделить компоненту фиксированного размера $n$, то мы возьмем вершины с наибольшим или наименьшим значением в собственном векторе. Если размер заранее не задан, то можно посмотреть на знак. 

In [ ]:
spectral_coloring = np.ravel(np.sign(f))

In [ ]:
plot_graph(Gmain, spectral_coloring)

In [ ]:
plot_graph(Gmain, [sum([1 for y in np.ravel(f) if x>y]) for x in np.ravel(f)], plt.cm.hsv)

Если, же нам нужно получить больше компонент, то рекомендуется использовать кластеризацию, например kmeans на значениях k собственных векторов, соответствующих наименьшим собственным числам, начиная со второго. На практике с этим иногда бывают проблеммы, так как в отдельные кластеры могут выделяться отдельные вершины, лежащие "в стороне".

In [ ]:
import scipy.cluster.vq as vq
k=3
means, labels = vq.kmeans2(U[:,1:k], k)
plot_graph(Gmain, labels)

Раньше мы смотрели на разделение, но можно подойти и по другому. Давайте считать вершины лежащими в одной компоненте, если они принадлежжат одной клике заданного размера.

In [ ]:
clique_components = list(community.k_clique_communities(Gmain, 6))

In [ ]:
def get_colors(G, comp):
    d={}
    c = 0
    for i in comp:
        c+=1
        for v in i:
            d[v]=c
    res = []
    for v in G.nodes:
        if v in d: res.append(d[v])
        else: res.append(0)
    return(res)

In [ ]:
clique_col = get_colors(Gmain, clique_components)

In [ ]:
plot_graph(Gmain, clique_col)

Давайте подумаем про хорошее разделение на комьюнити. Базовая идея хочется что бы любая вершина в "своей" компоненте имела связей не меньше чем с чужой, ну или хотя бы в среднем было так.

Определим модулярность, как $\sum\limits_i e_{ii} - a^2_i$, где $e_{ii}$ -- вероятность ребра лежать в компоненте $i$, а $a_i$ вероятность попасть в компоненту $i$ хотя бы одним ребром. 

Простейший алгоритм, реализуйте его:

1. Начинаем с произвольного разделения на несколько компонент.
2. Вычисляем улучшение модулярности при переносе каждой незаблокированной вершины в другой кластер.
3. Находим максимальную выгоду. Переносим вершину и блокируем ее от дальнейшего переноса.
4. Повторяем шаги 2-3 пока не исчезнут вершины улучшающие разделение.
5. Выводим результат.

Иногда имеет смысл разблокировать все вершины и повторить алгоритм.

In [ ]:
def greedy_modularity(G, Partitions):
    #your code
    return NewPartitions

Выделение комьюнити на основе выкидывания "центральных ребер".
На прошлом занятии мы говорили про центральность по посреднечеству. Её можно считать не только для вершин, но и для ребер.
Самые центральные ребра это те, через которые проходит наибольшее число путей. Давайте выкидывать самые популярные ребра и смотреть на компоненты связности.
Такой алгоритм называется Girvan-Newman.

In [ ]:
gnc = community.girvan_newman(Gmain)

Обратите внимание, что мы получили генератор. Реально выкидывание ребер будет происходить только, когда мы будем к нему обращаться. При этом каждый раз мы будем получать но одну больше компонент связности.  

In [ ]:
gnc_set2 = next(gnc)
gnc_color2 = get_colors(Gmain, gnc_set2)


In [ ]:
plot_graph(Gmain, gnc_color2)

In [ ]:
gnc_set3 = next(gnc)
gnc_color3 = get_colors(Gmain, gnc_set3)


In [ ]:
plot_graph(Gmain, gnc_color3)

И последний вариант который хотелось сегодня обсудить, это выделение комьюнити на основе кластеризации.

Давайте определим похожесть вершин и применим алгоритм иерархической кластеризации. После чего будем менять порог и получать разные варианты разделения графа на комьюнити. Такой метод хорош тем, что позволяет использовать еще и дополнительную информацию про сами вершины, а не только граф. Реализуйте разбиение на комьюнити используя в качестве похожести косинус угла между строками матрицы смежности. Что изменится если использовать квадрат матрицы смежности?